# 1. 导入包

In [2]:
import numpy as np  
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns
from pandas.core.frame import DataFrame
from scipy.fftpack import fft,ifft
from tqdm import tqdm
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.ticker import Locator
from matplotlib.ticker import MaxNLocator

# 2. 加载数据文件

In [3]:
dataset_name = "Toilet"
csv_filename = "toilet.csv"
df = pd.read_csv(csv_filename, names=["time", "us", "acc","ws","label"])
df.head(5)

FileNotFoundError: File b'toilet.csv' does not exist

# 3. 预处理数据

In [ ]:
acc_list = np.array(df["acc"])
acc_time = np.array(df["time"])
acc_us = np.array(df["us"])
acc_ms = [int(ms) for ms in acc_us/1000]
acc_list = acc_list*1000 
acc_mean = np.mean(acc_list)
acc_std  = np.std(acc_list)
acc_mean_str = "{:.2f}".format(acc_mean)
acc_std_str = "{:.2f}".format(acc_std)
print("the acc dist mean:%.2fmg and std:%.2fmg" %(acc_mean,acc_std))

# 4. 加速度传感器矫正

In [ ]:
# 加速度传感器校准用
CONST_STATIC_WINDOW = 64
static_mean_list = []
static_std_list  = []
#遍历加速度，寻找传感器安静的时间段，记录它的avg和std
for k in tqdm(range(CONST_STATIC_WINDOW,acc_list.size,1)):
    acc_list_slice = acc_list[k-CONST_STATIC_WINDOW:k]
    acc_mean = np.mean(acc_list_slice)
    acc_std  = np.std(acc_list_slice)
    if acc_std < 3:
        static_std_list.append(acc_std)
        static_mean_list.append(acc_mean)

# 5. 绘制加速度传感器在静止时候的均值-标准差分布

In [4]:
static_mean = np.mean(static_mean_list)
static_std = np.mean(static_std_list)
static_mean_str = "{:.2f}".format(static_mean)
static_std_str = "{:.2f}".format(static_std)
# print(static_std_list)
bins=np.arange(np.min(static_std_list),np.max(static_std_list),0.1)
plt.figure(figsize=(8, 8), dpi=100)
ax = plt.subplot()
ax.set_xlim(np.min(static_std_list), np.max(static_std_list))
sns.kdeplot(static_std_list,static_mean_list,shade=True)
plt.title("Sensor Base: mean="+static_mean_str+"mg std="+static_std_str+"mg")
plt.savefig(dataset_name+"_static.png")

NameError: name 'static_mean_list' is not defined

# 6. 矫正加速度列表

In [5]:
acc_list = acc_list - static_mean # 将加速度列表矫正到0

NameError: name 'acc_list' is not defined

# 7. 以秒为单位计算std和avg，找出最大的以判断时域波形刻度范围

In [6]:
SECOND_WINDOW = 32# sample rate 32
second_mean_list = []
second_std_list  = []
acc_mean_max = float()
acc_std_max = float()
#遍历加速度，寻找传感器安静的时间段，记录它的avg和std
for k in tqdm(range(SECOND_WINDOW,acc_list.size,SECOND_WINDOW)):
    acc_list_slice = acc_list[k-SECOND_WINDOW:k-1]
    acc_mean_max = acc_mean_max if(acc_mean_max>np.mean(acc_list_slice)) else np.mean(acc_list_slice)
    acc_std_max  = acc_std_max if(acc_std_max>np.std(acc_list_slice)) else np.std(acc_list_slice)
    second_mean_list.append(np.mean(acc_list_slice))
    second_std_list.append(np.std(acc_list_slice))

acc_mean_str = "{:.2f}".format(acc_mean_max)
acc_std_str = "{:.2f}".format(acc_std_max)
# print(static_std_list)
bins=np.arange(np.min(second_std_list),np.max(second_std_list),0.1)
plt.figure(figsize=(8, 8), dpi=100)
ax = plt.subplot()
ax.set_xlim(np.min(second_std_list), np.max(second_std_list))
sns.kdeplot(second_std_list,second_mean_list,shade=True)
plt.title("max_mean="+acc_mean_str+"mg max_std="+acc_std_str+"mg")
plt.savefig(dataset_name+"_dist_std_and_mean.png")

NameError: name 'acc_list' is not defined

# 8. 以1024个点为单位遍历一次，找FFT的直流分量大小

In [7]:
FFT = list()
for i in range(0,len(acc_list)):
#     if FFT_0<np.mean(acc_list[i:i+32]):
    FFT.append(np.mean(acc_list[i:i+1024]))
#     print(np.mean(acc_list[i:i+32]),i)
np.mean(FFT)

NameError: name 'acc_list' is not defined

# 9. 开始画图

In [8]:
fig= Figure(figsize=[25.6,14.4],dpi=100,facecolor="black",edgecolor="#BEBEBE")
canvas = FigureCanvas(fig)
ax_chart_32s = fig.add_axes([100/2560,970/1440,1250/2560,420/1440])
ax_chart_1s = fig.add_axes([1510/2560,970/1440,372/2560,420/1440])

ax_violin = fig.add_axes([1510/2560,510/1440,372/2560,412/1440])
ax_violins = fig.add_axes([100/2560,510/1440,1350/2560,412/1440])

ax_FFT = fig.add_axes([100/2560,50/1440,1350/2560,406/1440])

ax_kde_1 = fig.add_axes([1510/2560,50/1440,405/2560,405/1440])
ax_kde_2 = fig.add_axes([1975/2560,50/1440,531/2560,405/1440])

ax_video = fig.add_axes([1975/2560,476/1440,531/2560,944/1440])#[x,y,width,height] y in range of 456 and 496
canvas.print_figure("123.png")
fig.savefig("sample.png",facecolor=fig.get_facecolor(),edgecolor=fig.get_edgecolor())

In [9]:
CONST_ACC_MAX = math.pow(2, int(math.log(acc_mean_max + 4 * acc_std_max, 2)) + 1)
CONST_ACC_MAX

ValueError: math domain error

# 使用seaborn画violin

In [1]:
#plt.savefig("sample.png")
CONST_ACC_MAX = 32
# define the limits automatically
#设置时域图和violin的y坐标的上下限
CONST_ACC_MIN = -CONST_ACC_MAX
CONST_CHART_WINDOW = 1024
CONST_FFT_WINDOW = 1024
CONST_VIOLIN_WINDOW = 32
CONST_VIOLINS_WINDOW = 1024
CONST_VIOLINS_DIV = 32
CONST_SAMP_RATE = 32  # 采样率
CONST_COLOR_STD_LMT = 128
CONST_BLUE_RGB = [0, 0, 255]
CONST_WHITE_RGB = [255, 255, 255]
CONST_FFT_MIN = 0
CONST_FFT_MAX = 1
#可以从第几个帧开始绘制
CONST_MIN_K = 0
#帧图片文件夹
CONST_FRAMES_DIR = "dir_frames_toilet"

def whiten(ax, title):
    spine = ['bottom', 'top', 'right', 'left']
    for tickline in ax.xaxis.get_ticklines():
        tickline.set_color('white')
    for ticklabel in ax.xaxis.get_ticklabels():
        ticklabel.set_color('white')
    for tickline in ax.yaxis.get_ticklines():
        tickline.set_color('white')
    for ticklabel in ax.yaxis.get_ticklabels():
        ticklabel.set_color('white')
    for orien in spine:
        ax.spines[orien].set_color('black')
    ax.set_facecolor("black")  # grey
    title.set_color("white")
    title.set_fontsize('large')


def whiten_violin(result):
    for key in result:
        if key is not 'bodies':
            result[key].set_color('')
            
def violins_color_decide(color_relative):
    if color_relative<2:
        return [1,0,0]#red
    elif color_relative<4:
        return [255/255,128/255,0/255]#orange
    elif color_relative<8:
        return [255/255,255/255,0/255]#yellow
    elif color_relative<16:
        return [255/255,255/255,128/255]#light yellow
    elif color_relative<32:
        return [1,1,1]#WHITE
    elif color_relative<64:
        return [128/255,255/255,255/255]#light blue
    else:
        return [0,255/255,255]#plain blue
            
for k in tqdm(range(CONST_MIN_K, acc_list.size)):  # start form 1024, shift 8 every time
    #list.index()
    ax_violin.set_ylim(CONST_ACC_MIN, CONST_ACC_MAX)
    ax_violins.set_ylim(CONST_ACC_MIN, CONST_ACC_MAX)
    

    # 绘制时域32s图
    slice_start = k - CONST_FFT_WINDOW
    slice_end = k
    if slice_start < 0:
        slice_start = 0
    acc_slice = acc_list[slice_start:slice_end]
    avg_slice = acc_avg[slice_start:slice_end]
    std_slice = acc_std_list[slice_start:slice_end]
    acc_slice = np.lib.pad(acc_slice, (CONST_CHART_WINDOW - acc_slice.size, 0), 'constant', constant_values=(0, 0))
    avg_slice = np.lib.pad(avg_slice, (CONST_CHART_WINDOW - acc_slice.size, 0), 'constant', constant_values=(0, 0))
    std_slice = np.lib.pad(std_slice, (CONST_CHART_WINDOW - acc_slice.size, 0), 'constant', constant_values=(0, 0))

    ax_chart_32s.plot(acc_slice, 'w',avg_slice,"y")  # '--w'
    ax_chart_32s.set_ylabel("avg and realtime",color = "w")
    ax_chart_32s.set_ylim(CONST_ACC_MIN, CONST_ACC_MAX)
    title = ax_chart_32s.set_title("Realtime chart,Time: " + str(acc_time[k]) + "  Reading: " + "{:.2f}".format(acc_list[k]) + "mg")
    whiten(ax_chart_32s, title)
    
    #32s时域图上加std，双坐标系
    ax_chart_32s_std = ax_chart_32s.twinx()
    ax_chart_32s_std.plot(std_slice,"g")
    ax_chart_32s_std.set_ylabel("std",color = "w")
    ax_chart_32s_std.set_ylim(-acc_std_max,acc_std_max)
    whiten(ax_chart_32s_std, title)

    #绘制时域1s
    acc_slice_1s = acc_slice[-32:]
    ax_chart_1s.plot(acc_slice_1s,'w')
    ax_chart_1s.set_ylim(CONST_ACC_MIN, CONST_ACC_MAX)
    title = ax_chart_1s.set_title("Last one second")
    whiten(ax_chart_1s, title)
    
    # 绘制FFT图
    #########################################
    fft_abs = abs(fft(acc_slice))
    fft_norm = fft_abs / int(CONST_FFT_WINDOW)
    fft_half = fft_norm[:int(CONST_FFT_WINDOW / 2)]
    fft_half[0]=0

    ax_FFT.bar(x=np.linspace(0, CONST_SAMP_RATE / 2, 513)[:-1],  # the first 128 points covers frequency from 0-16hz
                   height=fft_half,width=0.032,bottom=0,align="center",color="white")

    ax_FFT.xaxis.set_major_locator(MaxNLocator(nbins=20, prune='lower'))
    title = ax_FFT.set_title("FFT chart:0 - 16Hz")
    whiten(ax_FFT, title)
    ax_FFT.set_ylim(CONST_FFT_MIN, CONST_FFT_MAX)
    ax_FFT.set_xlim(0, 16)  # 0-4hz

    # 绘制violin图
    ##########################################
    slice_start = k - CONST_VIOLIN_WINDOW
    slice_end = k
    if slice_start < 0:
        slice_start = 0
    acc_slice = acc_list[slice_start:slice_end]
    acc_slice = np.lib.pad(acc_slice, (CONST_VIOLIN_WINDOW - acc_slice.size, 0), 'constant', constant_values=(0, 0))
    acc_slice_dataset = pd.DataFrame(acc_slice)
    violin_mean = acc_slice_dataset[0].mean()
    violin_std = acc_slice_dataset[0].std()
    violin_mean_str = "{:.5f}".format(violin_mean)
    violin_std_str = "{:.5f}".format(violin_std)

    face_color = [(CONST_WHITE_RGB[i] - (CONST_WHITE_RGB[i] - CONST_BLUE_RGB[i]) * abs(violin_std / CONST_COLOR_STD_LMT)) / 255 for i in range(3)]
    for color in face_color:
        if color<0:
            color=0

    ###########seaborn violin#############
    violinplot_ax = sns.violinplot(x=acc_slice, color=face_color, ax=ax_violin, linewidth=0,orient="v")
    ax.grid(False)
    
    title = ax_violin.set_title("Realtime violin. Mean: " + violin_mean_str + "  Std: " + violin_std_str)
    ax_violin.yaxis.set_major_locator(MaxNLocator(nbins=16, prune='lower'))
    whiten(ax_violin, title)

    # 绘制violins图
    ################################
    slice_start = k - CONST_VIOLINS_WINDOW
    slice_end = k
    if slice_start < 0:
        slice_start = 0
    acc_slice = acc_list[slice_start:slice_end]
    acc_slice = np.lib.pad(acc_slice, (CONST_VIOLINS_WINDOW - acc_slice.size, 0), 'constant', constant_values=(0, 0))

    violins_dataset = pd.DataFrame([acc_slice[int(t - CONST_VIOLINS_DIV):int(t)] for t in list(np.linspace(0, 1024, CONST_VIOLINS_DIV + 1)[1:])])
    
    color_list = []
    for row in range(CONST_VIOLINS_DIV):
        violins_color = abs(violins_dataset.loc[row, :].std())
        violins_color_relative = violins_color / CONST_COLOR_STD_LMT
        if violins_color_relative>1:
            violins_color_relative=1
        color_list.append(violins_color_decide(violins_color_relative*128))
        
    violins_result = ax_violins.violinplot(dataset=violins_dataset)
    ax_violins.yaxis.set_major_locator(MaxNLocator(nbins=(CONST_ACC_MAX / 4) + 1, prune='lower'))
    
    for pc in range(len(violins_result['bodies'])):
        violins_result['bodies'][pc].set_facecolor(color_list[pc])
        violins_result['bodies'][pc].set_edgecolor(color_list[pc])
        violins_result['bodies'][pc].set_alpha(1.0)
    
    title = ax_violins.set_title("History violins")
    ax_violins.yaxis.set_major_locator(MaxNLocator(nbins=16, prune='lower'))
    whiten(ax_violins, title)
    whiten_violin(violins_result)
    
    # 调整 video框
    ax_video.set_facecolor("black")
    spine = ['bottom', 'top', 'right', 'left']
    for orien in spine:
        ax_video.spines[orien].set_color('white')
    ax_video.axis(option=False)

    # save figs
    fig.savefig("dir_frames_printer/" + str(10000000000 + acc_ms[k]) + '.png', facecolor=fig.get_facecolor(), edgecolor='none')
    ax_chart_32s.cla()
    ax_chart_32s_std.cla()
    ax_chart_1s.cla()
    ax_FFT.cla()
    ax_violin.cla()
    ax_violins.cla()


NameError: name 'tqdm' is not defined